In [21]:
import sys
path = r'C:\Users\infin\OneDrive\Documents\GitHub\aimodels\projects\tooling'
sys.path.insert(0, path)
from mixers import MixerModel 
from data import WrappedHuggingFaceTokenizer

In [2]:
import datasets
simple = "rahular/simple-wikipedia"
simple = datasets.load_dataset(simple)
simple

Found cached dataset parquet (C:/Users/infin/.cache/huggingface/datasets/rahular___parquet/rahular--simple-wikipedia-e8322ace21be7483/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 769764
    })
})

In [3]:
def quick_estimate_tokens(ds, field="text", already_tokenized = False, chars_per_token=4):
    tally = 0
    max_len = 0
    lengths = {}
    for row in ds:
        l = len(row[field])
        tally += l
        lengths[l] = lengths.get(l, 0) + 1
        if l > max_len:
            max_len = l

    print(f'{int(tally):_}')
    print(f'Max length: {max_len}, estimated tokens: {int(max_len / chars_per_token):_}')
    lengths = list(lengths.items())
    lengths.sort(reverse=True)
    return int(tally), lengths

total, length = quick_estimate_tokens(simple['train'])


141_248_640
Max length: 10570, estimated tokens: 2_642


In [4]:
import pandas as pd
df = pd.DataFrame(length, columns=['Length', 'Count'])
#histogram
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
def map_tokenize_as_list(x):
    return tokenizer(x['text'])



list_tokenized = simple['train'].map(map_tokenize_as_list, batched=True, remove_columns=['text']) # That takes 30 seconds to tokenize



Map:   0%|          | 0/769764 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1340 > 1024). Running this sequence through the model will result in indexing errors


In [18]:
list_tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 769764
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token
def map_tokenize_as_tensors(x):
    return tokenizer(x['text'], return_tensors="pt", padding=True, truncation=True, max_length=512)

tensor_tokenized = simple['train'].map(map_tokenize_as_tensors, batched=True, remove_columns=['text']) # That takes 2 whole minues to tokenize

Map:   0%|          | 0/769764 [00:00<?, ? examples/s]

In [20]:
import time
start = time.time()
for batch in tokenizing_dataloader:
    pass
print(time.time() - start)

TypeError: list indices must be integers or slices, not str

In [15]:
quick_estimate_tokens(tokenized['train'], field="input_ids")

: 

In [8]:
def token_profile(ds, field, print_every = None)
    """ This should operate on an already-tokenized dataset. """
    freqs = {}
    # can we do this with datasets.map?
    n = len(ds)
    print(f"Processing {n} examples")
    for idx, ex in enumerate(ds):
        if print_every is not None and idx % print_every == 0:
            print(f"Processed {idx} examples")

        if 
        for t in tokens:
            print(t)
            if t in freqs:
                freqs[t] += 1
            else:
                freqs[t] = 1

        break

    total = sum(freqs.values())
    desc = [(tknzr.decode(k), v/total) for k, v in freqs.items()]
    return total, desc, freqs


import datasets
simple = "rahular/simple-wikipedia"
simple = datasets.load_dataset(simple)
simple

from data import WrappedHuggingFaceTokenizer
tokenizer = WrappedHuggingFaceTokenizer(WrappedHuggingFaceTokenizer.MISTRAL)

total, ordered, freqs = token_profile(simple['train'], 'text', tokenizer, print_every = 100_000)

Found cached dataset parquet (C:/Users/infin/.cache/huggingface/datasets/rahular___parquet/rahular--simple-wikipedia-e8322ace21be7483/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Processing 769764 examples
Processed 0 examples
torch.Size([1, 2])
tensor([   1, 3999])


In [6]:
freqs

{tensor([   1, 3999]): 1}

In [ ]:
# Okay.  So, question.  Attention mask.  Ah okay, so Hugging Face expects the data collator to take care of the masking.
# And I guess the reason so many tokenizers don't have default pad_tokens is that they were trained without them and Hugging Face wants to serve up exactly what was used in training.
# So now I understand "why the fence is there"; now how do I change it?
# So there's a bit of a dilemma here because I want to be able to use Hugging Face's tokenizers, but I don't want to depend on their overall data collation strategy.
# I could wrap their tokenizer in my own class that changes eos_token to -100.  Ummm...will the embedding choke on that?  I believe it will.  Well, a pretrained embedding will, anyway.
# There's going to be a potentially serious issue here where comparisons to pretrained models are really hard.  But maybe I can set up a wrapper for that.
# So, question about padding strategies.  I'm currently using the nn.Transformers method that creates causal attention masks.
# One thing I could just do is assume -100 is always the padding token.  If I do that, then I also have to...well, hold on.  If we left-pad, then things get weird and I have to use an attention mask.
# But I think if I right-pad, everything is fine?  You'll waste a bit of time calculating attention for the padded tokens in batches where not eveyrthing is the same length, but I think you have to do that anyway.
# I think left-padding is a strategy for .generate()?  I think that's it.
